In [1]:
import torch
import torch.nn as n
import torch.nn.functional as F
import torch.utils.data
import torch.optim as optim
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import time

In [2]:
def read_images(path,num_img):
    array = np.zeros([num_img,64*32])
    i = 0
    for img in os.listdir(path):
        img_path = path + "//" + img
        img = Image.open(img_path, mode="r")
        data = np.asarray(img,dtype = "uint8")
        data = data.flatten()
        array[i,:]=data
        i += 1
    return array

In [3]:
#train negative
train_negative_path = r"../input/lsifir/LSIFIR/Classification/Train/neg"
num_train_negative_img = 43390
train_negative_array = read_images(train_negative_path, num_train_negative_img)

x_train_negative_tensor = torch.from_numpy(train_negative_array)
print("x_train_negative_tensor : ",x_train_negative_tensor.size())
y_train_negative_tensor = torch.zeros(num_train_negative_img, dtype = torch.long)
print("y_train_negative_tensor : ", y_train_negative_tensor.size())

x_train_negative_tensor :  torch.Size([43390, 2048])
y_train_negative_tensor :  torch.Size([43390])


In [4]:
#train positive
train_positive_path = r"../input/lsifir/LSIFIR/Classification/Train/pos"
num_train_positive_img = 10208
train_positive_array = read_images(train_positive_path,num_train_positive_img)

x_train_positive_tensor = torch.from_numpy(train_positive_array)
print("x_train_positive_tensor : ",x_train_negative_tensor.size())
y_train_positive_tensor = torch.zeros(num_train_negative_img, dtype = torch.long)
print("y_train_positive_tensor : ", y_train_positive_tensor.size())

x_train_positive_tensor :  torch.Size([43390, 2048])
y_train_positive_tensor :  torch.Size([43390])


In [5]:
#train concat
x_train = torch.cat((x_train_negative_tensor, x_train_positive_tensor),0)
y_train = torch.cat((y_train_negative_tensor, y_train_positive_tensor),0)
print("x_train : ", x_train.size())
print("y_train : ", y_train.size())

x_train :  torch.Size([53598, 2048])
y_train :  torch.Size([86780])


In [6]:
#test negative
test_negative_path = r"../input/lsifir/LSIFIR/Classification/Test/neg"
num_test_negative_img = 22050
test_negative_array = read_images(test_negative_path,num_test_negative_img)

x_test_negative_tensor = torch.from_numpy(test_negative_array[:20855,:])
print("x_test_negative_tensor : ",x_test_negative_tensor.size())
y_test_negative_tensor = torch.zeros(20855, dtype = torch.long)
print("y_test_negative_tensor : ", y_test_negative_tensor.size())

x_test_negative_tensor :  torch.Size([20855, 2048])
y_test_negative_tensor :  torch.Size([20855])


In [7]:
#test positive
test_positive_path = r"../input/lsifir/LSIFIR/Classification/Test/pos"
num_test_positive_img = 5944
test_positive_array = read_images(test_positive_path,num_test_positive_img)

x_test_positive_tensor = torch.from_numpy(test_positive_array)
print("x_test_positive_tensor : ",x_test_negative_tensor.size())
y_test_positive_tensor = torch.zeros(num_test_negative_img, dtype = torch.long)
print("y_test_positive_tensor : ", y_test_positive_tensor.size())

x_test_positive_tensor :  torch.Size([20855, 2048])
y_test_positive_tensor :  torch.Size([22050])


In [8]:
#concat test
x_test = torch.cat((x_test_negative_tensor, x_test_positive_tensor),0)
y_test = torch.cat((y_test_negative_tensor, y_test_positive_tensor),0)
print("x_test : ", x_test.size())
print("y_test : ", y_test.size())

x_test :  torch.Size([26799, 2048])
y_test :  torch.Size([42905])


In [9]:
#CNN Hyperparameters

num_epochs = 5000
num_class = 2
batch_size = 8933
learning_rate = 0.00001

In [10]:
class Net(n.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = n.Conv2d(1,10,5)
        self.pool = n.MaxPool2d(2,2)
        self.conv2 = n.Conv2d(10,16,5)
        self.fc1 = n.Linear(16*13*5,520)
        self.fc2 = n.Linear(520,130)
        self.fc3 = n.Linear(130,num_class)
        
        
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
        x = x.view(-1,16*13*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [11]:
train = torch.utils.data.TensorDataset(x_train[:53598,:2048], y_train[:53598])
trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle = True)

In [12]:
test = torch.utils.data.TensorDataset(x_test[:26799, :2048], y_test[:26799])
testloader = torch.utils.data.DataLoader(test, batch_size, shuffle = False)

In [13]:
net = Net()

In [14]:
criterion = n.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=learning_rate, momentum = 0.8)

In [16]:
start = time.time()
train_acc = []
test_acc = []
loss_list = []

for epoch in range(num_epochs):
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.view(batch_size,1,64,32)
        inputs = inputs.float()
        
        #zero graient
        optimizer.zero_grad()
        
        #forward
        outputs = net(inputs)
        
        #loos
        loss = criterion(outputs,labels)
        
        #back
        loss.backward()
        
        #update weights
        optimizer.step()
        
    #test
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images.view(batch_size,1,64,32)
            images = images.float()
            
            outputs = net(images)
            _,predicted = torch.max(outputs.data,1)
            
            total += labels.size(0)
            
            correct += (predicted == labels).sum().item()
    
    acc1 = 100*correct/total
    print("accuracy test : ", acc1)
    
    test_acc.append(acc1)
    
    #train
    correct = 0
    total = 0
    with torch.no_grad():
        for data in trainloader:
            images, labels = data
            images.view(batch_size,1,64,32)
            images = images.float()
            
            outputs = net(images)
            _,predicted = torch.max(outputs.data,1)
            
            total += labels.size(0)
            
            correct += (predicted == labels).sum().item()
    
    acc2 = 100*correct/total
    print("accuracy train : ", acc2)
    
    train_acc.append(acc2)
    
print("train is done")
        
        


#train





end = time.time()
process_time = (end-start)/60
print("Process Time : ",process_time)

NameError: name 'view' is not defined